<a href="https://colab.research.google.com/github/AndresFloresF/DashCursoGeo/blob/master/DashSylvereyeExampleActivityNotDone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo Dash Sylvereye: 
### Visualizando las longitudes de calles
### Autor: Alberto García Robledo, CONACYT-CentroGeo


In [1]:
# instalar Dash Sylvereye
# esto puede tomar un tiempo

!pip install --extra-index-url http://pypi.observatoriogeo.mx/simple/ --trusted-host pypi.observatoriogeo.mx dash_sylvereye==0.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, http://pypi.observatoriogeo.mx/simple/
     |████████████████████████████████| 2.1 MB 9.8 kB/s 
     |████████████████████████████████| 66 kB 3.8 MB/s 
     |████████████████████████████████| 1.9 MB 41.5 MB/s 
     |████████████████████████████████| 6.3 MB 53.6 MB/s 
     |████████████████████████████████| 41 kB 216 kB/s 
     |████████████████████████████████| 1.2 MB 42.9 MB/s 
     |████████████████████████████████| 3.4 MB 41.1 MB/s 
     |████████████████████████████████| 187 kB 51.0 MB/s 
     |████████████████████████████████| 1.8 MB 64.1 MB/s 
     |████████████████████████████████| 357 kB 48.8 MB/s 
  Created wheel for dash-sylvereye: filename=dash_sylvereye-0.1.1-py3-none-any.whl size=2132599 sha256=23282b4db5eb545162cf65c76e762a920ccb2bbe7da9f9db26d359b3371e7f40
  Stored in directory: /root/.cache/pip/wheels/da/7e/de/83ab15285e8ee7e3465364220479c6445e895b8ffd0b9f89ff
  Created w

In [2]:
# instalar otras dependencias
# esto puede tomar un tiempo

!pip install osmnx
!pip install jupyter-dash
!pip install dash_bootstrap_components==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 1.0 MB 66.8 MB/s 
     |████████████████████████████████| 1.0 MB 40.0 MB/s 
     |████████████████████████████████| 11.2 MB 45.0 MB/s 
     |████████████████████████████████| 16.7 MB 51.3 MB/s 
     |████████████████████████████████| 944 kB 48.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=cb7001f941bf19d7ad7167512f1602eeb9de2f8fa1a2f53cb27d7a368cecb4ec
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 14.8 MB/s 


In [3]:
# importar clases y métodos a usar

import osmnx as ox
import numpy as np
import pandas
from jupyter_dash import JupyterDash
from dash_html_components import Div
from dash_sylvereye import SylvereyeRoadNetwork
from dash_sylvereye.utils import load_from_osmnx_graph 
from dash_sylvereye.enums import NodeSizeMethod, EdgeColorMethod, EdgeWidthMethod, EdgeAlphaMethod
from dash_sylvereye.defaults import get_default_node_options, get_default_edge_options
from dash.dependencies import Input, Output
from dash_html_components import H2, H3
from dash_core_components import Markdown, Graph
import dash_bootstrap_components as dbc

In [10]:
# configuración de la aplicación

OSMNX_QUERY = 'Iztapalapa'
TILE_LAYER_URL = '//stamen-tiles-{s}.a.ssl.fastly.net/toner/{z}/{x}/{y}.png'
TILE_LAYER_SUBDOMAINS = 'abcd'
TILE_LAYER_ATTRIBUTION = 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.'
MAP_CENTER = [19.3429,-99.0502]
MAP_ZOOM = 15
MAP_STYLE = {'width': '100%', 'height': '100vh'}
TILE_LAYER_OPACITY = 0.20

In [11]:
# obtener la topología de la red de caminos desde OSM 
# esto puede tomar un tiempo

road_network = ox.graph_from_place(OSMNX_QUERY, network_type='drive') 
_, edges_data = load_from_osmnx_graph(road_network)

In [12]:
# asignar las longitudes de las calles como pesos

for edge in edges_data: 
  edge["data"]["weight"] = edge["data"]["length"]


In [13]:
# configurar las opciones de visualización de los enlaces

edge_options = get_default_edge_options()
edge_options["width_method"] = EdgeWidthMethod.SCALE
edge_options["width_scale_field"] = "weight"
edge_options["color_method"] = EdgeColorMethod.SCALE
edge_options["color_scale_field"] = "weight"
edge_options["color_scale_left"] = 0xcbdbff
edge_options["color_scale_right"] = 0x06696
edge_options["alpha_method"] = EdgeAlphaMethod.SCALE
edge_options["alpha_scale_field"] = "weight"

In [14]:
# construir visualización

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

navbar = dbc.NavbarSimple(
    brand="Visualization of street lengths with Dash Sylvereye",
    brand_href="#",
    color="primary",
    dark=True,
)

sylvereye = \
    SylvereyeRoadNetwork(
                         id='sylvereye-roadnet',
                         tile_layer_url=TILE_LAYER_URL,
                         tile_layer_subdomains=TILE_LAYER_SUBDOMAINS,
                         tile_layer_attribution=TILE_LAYER_ATTRIBUTION,
                         map_center=MAP_CENTER,
                         map_zoom=MAP_ZOOM,
                         map_style=MAP_STYLE,                         
                         edges_data=edges_data,                         
                         edge_options=edge_options,                         
                         tile_layer_opacity=TILE_LAYER_OPACITY
                        )
markdown = \
  Markdown(id='markdown-street-data')
 
app.layout = dbc.Container(
    [
        navbar,
        dbc.Row(
            [
                dbc.Col(sylvereye, md=10),
                dbc.Col(markdown, md=2)
            ],
            align='center'
        )       
    ], fluid=True
)
@app.callback(
    Output('markdown-street-data', 'children'),
    [Input('sylvereye-roadnet', 'clicked_edge')])
def update_street_data(clicked_edge):        
    if clicked_edge:

        data = clicked_edge["data"]["data"]
               
        return f'''

        ## Query

        {OSMNX_QUERY}

        ## Street data

        | Property | Value |
        | -------- | ----- |
        | Source OSMID | {data["source_osmid"]} |
        | Target OSMID | {data["target_osmid"]} |
        | Street name| {data["name"]} |
        | Max. speed | {data["maxspeed"]} |
        | Length | {data["length"]} |    
        | One-way | {data["oneway"]} |
        
        '''


In [15]:
# correr visualización
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>